# The MRI Signal Equation

The MRI signal equation can be used to understand how images are formed as well as the influence of other factors on contrast, image quality and artifacts.  It is derived from the Bloch equation and can incorporate the effects of RF coils and other in vivo spin physics.
The signal equation can be simplified in many ways, which we will use throughout this book.  Here we build up a comprehensive MRI signal equation. 

## Learning Goals

1. Understand what MRI is measuring
   * Describe what produces the MRI signal
   * Describe how gradients influence MRI signal
   * Describe how in vivo spin physics phenomena influence MRI signal

## The Transverse Magnetization

The signal in MRI comes from the precession of the transverse component of the net magnetization, which creates a time-varying magnetic field that induces a voltage in the RF receive coil.  The signal comes from the magnetization across a large volume:

$$s(t) = \int M_{XY}(\vec{r},t) \  d\vec{r}$$

For simplifaction, we use the notation

$$m(\vec{r}) = M_{XY}(\vec{r}, t=0)$$

and then assume the transverse magnetization does not change in time (don't worry, we will add this back in below), we get:

$$s(t) = \int m(\vec{r}) \  d\vec{r}$$

Here we also use the rotating frame.

## Adding Relaxation

The transverse magnetization decays due to relaxation, which can be added to the signal equation:

$$s(t) = \int m(\vec{r})\ e^{-t/T_2^*(\vec{r})}\  d\vec{r}$$

## Adding Off-resonance and Chemical Shift

Off-resonance and chemical shift lead to changes in the precession frequency of the magnetization.  We can add this onto the idealized signal equation.

Here we separately define:
* $\Delta f_r(\vec{r}) = \bar \gamma \Delta B_0(\vec{r})$ - magnetic field inhomogeneities, including main field imperfections, and magnetic susceptibility effects
* $\Delta f_{cs}$ - frequency changes due to chemical shift, so depends on the chemical species being examined (could sum or index signal over this dimension)


<!-- $$s(t) = \int m(\vec{r})\ e^{-i 2 \pi \Delta f_{cs} t} e^{-i 2 \pi \Delta f_r(\vec{r}) t} e^{-i 2 \pi \vec{k}(t) \cdot \vec{r}} \  d\vec{r}$$ -->
For simplicity, combine all off-resonance and/or chemical shift into a single frequnecy shift term, $\Delta f(\vec{r}) = \Delta f_{cs} + \Delta f_r(\vec{r})$ as

$$s(t) = \int m(\vec{r})\ e^{-i 2 \pi \Delta f(\vec{r}) t} \  d\vec{r}$$

## Adding RF Coil Profiles

The RF receive coils are not equally sensitive to the entire volume, but rather have a sensitivity profile that is spatially varying.  For example, they are typically more sensitive to the precession of the transverse magnetization near the coil and less sensitive to the magnetization further away.  

This sensitivity profile can be incorporated into the signal equation.  The fact that MRI usually uses multiple receive elements is added as the received signal for coil element $n$, which has a coil sensitivity profile $C_n(\vec{r})$.

$$s_n(t) = \int m(\vec{r})\ C_n(\vec{r}) \  d\vec{r}$$

## Adding Gradients

Magnetic field gradients create rotations in the transverse magnetization, and can be added to the signal equation:

$$s(t) = \int m(\vec{r}) \  \exp\left( -i \gamma \int_0^t \vec{G}(\tau) \cdot \vec{r} \ d\tau \right) \  d\vec{r}$$

The effect of gradients depends on their cumulative area, as described by the integral in the equation.

## Complete Signal Equation

Finally, we can write the most intimidating version, a comprehensive version of signal equation, including relaxation, coil sensitivity, off-resonance and chemical shift, and gradients.
The signal from an individual RF coil element $n$ is

$$s_n(t) = \int m(\vec{r})\ C_n(\vec{r})\ e^{-t/T_2^*(\vec{r})}\ e^{-i 2 \pi \Delta f_{cs} t} e^{-i 2 \pi \Delta f_r(\vec{r}) t} \  \exp\left( -i \gamma \int_0^t \vec{G}(\tau) \cdot \vec{r} \ d\tau \right) \  d\vec{r}$$


## Simplified Versions of the Signal Equation

For purposes including developing intuition, designing pulse sequences, formulating the image reconstruction, and understanding artifacts and imperfections, we use simplified versions of the signal equation.  A few useful simplifications are described below.

### Image Formation with only Gradients

An idealized signal equation only capturing the effects of gradients is commonly used, particularly for understanding image formation.  It neglects relaxation, RF coil profile, and off-resonance

$$s(t) = \int m(\vec{r}) \  \exp\left( -i \gamma \int_0^t \vec{G}(\tau) \cdot \vec{r} \ d\tau \right) \  d\vec{r}$$

### Relaxation, Off-resonance, and Chemical Shift in Images

Often the effects of the gradients and RF coil volume are neglected for understanding the effects of relaxation and off-resonance on the signal.

$$s(t) = m(\vec{r})\ e^{-t/T_2^*(\vec{r})}\ e^{-i 2 \pi \Delta f(\vec{r}) t} \  $$

When we perform proper spatial encoding and image reconstruction the effect of gradients and the integration over a volume are removed, and the resulting image is proportional to this simplified version of the signal equation.
